### Подготовка датасета для разведочного анализа
Загрузим полученный на этапе первичного сбора данных датасет и посмотрим на состав столбцов в нем

In [25]:
import pandas as pd
df = pd.read_csv('../data_collection/datasets/train.csv')
df

,Unnamed: 0,id,atm_group,address,address_rus,lat,long,target,area_fias_id,area_with_type,...,metro_distance_2,metro_station_name_3,metro_line_name_3,metro_distance_3,street,house_fias_id,house,metro,locality_area,locality_population
0,2606,5.0,496.5,BUDENNOGO 7A ELISTA,"улица С.М. Будённого, 7А, Элиста, Республика К...",46.318213,44.260407,0.019958,NaN,NaN,...,NaN,NaN,NaN,NaN,им Буденного С.М.,cc2b04c8-b15f-41c1-a9a2-e57ccd4c293b,7А,NaN,53.376920,103122.0
1,2607,6.0,496.5,"HO CHI MIHN AVE, 19 ULYANOVSK","г Ульяновск, пр-кт Хо Ши Мина, д 19",54.270398,48.300632,-0.089111,NaN,NaN,...,NaN,NaN,NaN,NaN,Хо Ши Мина,5780b18d-d67c-4300-b932-6104b2346faf,19,NaN,326.723424,627705.0
2,5126,7.0,496.5,SHELESTA 116A KHABAROVSK,"улица Шелеста, 116А, Хабаровск, Россия, 680042",48.520503,135.052576,-0.048769,NaN,NaN,...,NaN,NaN,NaN,NaN,Шелеста,933fb2ef-5e12-4dcb-a310-7e1e38f519d7,116А,NaN,389.085968,616372.0
3,2608,8.0,496.5,ORDZHONIKIDZE 52 YAKUTSK,"улица Орджоникидзе, 52, Якутск, Республика Сах...",62.025583,129.721362,-0.041545,NaN,NaN,...,NaN,NaN,NaN,NaN,Орджоникидзе,113d873b-ead4-42b2-acf8-d2d3ac427b5e,52,NaN,144.141162,322987.0
4,2609,10.0,496.5,"VETERANOV AVE, 3 KRASNOKAMENS","проспект Ветеранов, 11, Краснокаменск, Забайка...",50.089050,118.027310,0.025479,539e012d-a5c0-4a81-83b9-cf87ea0f7697,Краснокаменский р-н,...,NaN,NaN,NaN,NaN,Ветеранов,2e866034-f3cf-4015-8226-dfde4456bc1b,11,NaN,26.359358,51387.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,5125,8806.0,496.5,CHKALOVA 14 S. TROITSKOY,"улица Чкалова, 14, Шемуршинский район, Чувашск...",54.879459,47.574161,-0.097043,144ddb64-8ff6-4396-8ee6-7207bfbde842,Шемуршинский р-н,...,NaN,NaN,NaN,NaN,Чкалова,a94ade96-1e5a-45cb-bad4-da6ad4656107,14,NaN,18392.485727,1217818.0
6257,1368,8807.0,496.5,OKTYABRSKAYA 18A IKI-BURUL,"Октябрьская улица, 16, поселок Ики-Бурул, Ики-...",45.826910,44.647851,-0.090848,83386818-e69a-42c1-8695-dafc116ab823,Ики-Бурульский р-н,...,NaN,NaN,NaN,NaN,Октябрьская,772d5a34-b6dc-4900-88ef-67f212cff5d8,16,NaN,72104.779759,271135.0
6258,2605,8809.0,496.5,"ZHIGULSKOGO 1""Z"" LAGAN","улица Жигульского, 1, Лагань, Республика Калмы...",45.389283,47.361688,0.036916,92c30b00-187a-4d5e-9297-49366761187d,Лаганский р-н,...,NaN,NaN,NaN,NaN,Жигульского,a6895079-bb11-47f7-a7d4-3284fc0d1b45,1,NaN,11.643585,12769.0
6259,3873,8810.0,496.5,GORODOVIKOVA 3A ELISTA,"улица Б. Городовикова, 3А, Элиста, Республика ...",46.306940,44.266534,-0.135684,NaN,NaN,...,NaN,NaN,NaN,NaN,им Б.Б.Городовикова,bf58dd0f-fb55-4b1e-8169-2c5ceb30928e,1,NaN,53.376920,103122.0


Видно, что в нем есть поля, которые в дальнейшем не понадобятся: 
- поля с идентификаторами Федеральной информационной адресной системы (ФИАС). Никакой смысловой нагрузки они не несут и нужны только для того, чтобы однозначно идентифицировать элементы адресного деления внутри ФИАС. Эти идентификаторы могут пригодится для сбора дополнительных данных, но для анализа они не требуются
- поля с координатами элементов адресного деления ФИАС (региона, района, города, поселения и улицы). У нас уже есть более точные координаты расположения банкомата
- fias-level - уровень найденного элемента в системе ФИАС (уровень дома или только уровень улицы; или же вообще уровень района города). Этот уровень и так отражен в найденных компонентах адреса

Сформируем датасет без ненужных полей, а для нескольких оставшихся - сделаем переименование

In [27]:
final_col_list = [
    'Unnamed: 0', 'id', 'atm_group', 'address', 'address_rus', 'lat', 'long', 'target', 
    'federal_district', 
    'region_with_type',  #'region_fias_id', 'region_lat', 'region_lon',
    'area_with_type', #'area_fias_id', 'area_lat', 'area_lon', 
    'city_with_type', #'city_fias_id', 'city_lat', 'city_lon',
    'city_area',
    'city_district_with_type', #'city_district_fias_id', 
    'capital_marker', 
    'settlement_with_type', #'settlement_fias_id', 'settlement_lat', 'settlement_lon',
    'street_with_type', #'street_fias_id', 'street_lat', 'street_lon', 'street',
    'house',  #'house_fias_id', 
    'metro',
    'metro_station_name_1', 'metro_line_name_1', 'metro_distance_1', 
    'metro_station_name_2', 'metro_line_name_2', 'metro_distance_2', 
    'metro_station_name_3', 'metro_line_name_3', 'metro_distance_3', 
    'locality_area', 
    'locality_population', 
    #'fias_level', 'fias_id'
]

df = df[final_col_list]
df.rename(columns={'Unnamed: 0':'id_0', 'metro':'nearest_metro_distance'}, inplace=True)
df

,id_0,id,atm_group,address,address_rus,lat,long,target,federal_district,region_with_type,...,metro_line_name_1,metro_distance_1,metro_station_name_2,metro_line_name_2,metro_distance_2,metro_station_name_3,metro_line_name_3,metro_distance_3,locality_area,locality_population
0,2606,5.0,496.5,BUDENNOGO 7A ELISTA,"улица С.М. Будённого, 7А, Элиста, Республика К...",46.318213,44.260407,0.019958,Южный,Респ Калмыкия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.376920,103122.0
1,2607,6.0,496.5,"HO CHI MIHN AVE, 19 ULYANOVSK","г Ульяновск, пр-кт Хо Ши Мина, д 19",54.270398,48.300632,-0.089111,Приволжский,Ульяновская обл,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,326.723424,627705.0
2,5126,7.0,496.5,SHELESTA 116A KHABAROVSK,"улица Шелеста, 116А, Хабаровск, Россия, 680042",48.520503,135.052576,-0.048769,Дальневосточный,Хабаровский край,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,389.085968,616372.0
3,2608,8.0,496.5,ORDZHONIKIDZE 52 YAKUTSK,"улица Орджоникидзе, 52, Якутск, Республика Сах...",62.025583,129.721362,-0.041545,Дальневосточный,Респ Саха /Якутия/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.141162,322987.0
4,2609,10.0,496.5,"VETERANOV AVE, 3 KRASNOKAMENS","проспект Ветеранов, 11, Краснокаменск, Забайка...",50.089050,118.027310,0.025479,Дальневосточный,Забайкальский край,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.359358,51387.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,5125,8806.0,496.5,CHKALOVA 14 S. TROITSKOY,"улица Чкалова, 14, Шемуршинский район, Чувашск...",54.879459,47.574161,-0.097043,Приволжский,Чувашская Республика - Чувашия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18392.485727,1217818.0
6257,1368,8807.0,496.5,OKTYABRSKAYA 18A IKI-BURUL,"Октябрьская улица, 16, поселок Ики-Бурул, Ики-...",45.826910,44.647851,-0.090848,Южный,Респ Калмыкия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72104.779759,271135.0
6258,2605,8809.0,496.5,"ZHIGULSKOGO 1""Z"" LAGAN","улица Жигульского, 1, Лагань, Республика Калмы...",45.389283,47.361688,0.036916,Южный,Респ Калмыкия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.643585,12769.0
6259,3873,8810.0,496.5,GORODOVIKOVA 3A ELISTA,"улица Б. Городовикова, 3А, Элиста, Республика ...",46.306940,44.266534,-0.135684,Южный,Респ Калмыкия,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.376920,103122.0


Полученный датасет сохраним для проведения разведывательного анализа данных

In [28]:
df.to_csv('datasets/train.csv', index=False)